<a href="https://colab.research.google.com/github/jaredzrks0/DSAN5400_final_project/blob/main/DSAN_5400_Final_News_Recommender_Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os

In [2]:
# Install ColBERT and add to path

!git -C ColBERT/ pull || git clone https://github.com/stanford-futuredata/ColBERT.git > /dev/null 2>&1
import sys; sys.path.insert(0, 'ColBERT/')

fatal: cannot change to 'ColBERT/': No such file or directory


In [3]:
# Download the ColBERT Checkpoint
!wget "https://downloads.cs.stanford.edu/nlp/data/colbert/colbertv2/colbertv2.0.tar.gz"
!mkdir -p checkpoints  # Create the 'checkpoints' directory if it doesn't exist
!tar -xvzf colbertv2.0.tar.gz -C checkpoints

--2024-12-09 20:52:43--  https://downloads.cs.stanford.edu/nlp/data/colbert/colbertv2/colbertv2.0.tar.gz
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 405924985 (387M) [application/octet-stream]
Saving to: ‘colbertv2.0.tar.gz’

colbertv2.0.tar.gz  100%[===================>] 387.12M  5.07MB/s    in 72s     

2024-12-09 20:53:55 (5.36 MB/s) - ‘colbertv2.0.tar.gz’ saved [405924985/405924985]

colbertv2.0/
colbertv2.0/artifact.metadata
colbertv2.0/vocab.txt
colbertv2.0/tokenizer.json
colbertv2.0/special_tokens_map.json
colbertv2.0/tokenizer_config.json
colbertv2.0/config.json
colbertv2.0/pytorch_model.bin


In [4]:
try: # When on google Colab, let's install all dependencies with pip.
    import google.colab
    !pip install -U pip > /dev/null 2>&1
    !pip install fsspec==2024.9.0 > /dev/null 2>&1
    !pip install -e ColBERT/['faiss-gpu','torch'] > /dev/null 2>&1
    !pip install --upgrade torch torchvision torchaudio > /dev/null 2>&1
except Exception:
  import sys; sys.path.insert(0, 'ColBERT/')
  try:
    from colbert import Indexer, Searcher
  except Exception:
    print("If you're running outside Colab, please make sure you install ColBERT in conda following the instructions in our README. You can also install (as above) with pip but it may install slower or less stable faiss or torch dependencies. Conda is recommended.")
    assert False

In [5]:
# ColBERT Imports
import colbert
from colbert import Indexer, Searcher
from colbert.infra import Run, RunConfig, ColBERTConfig
from colbert.data import Queries, Collection

In [36]:
import pandas as pd
import logging

article_df = pd.read_csv('articles_with_text.csv')
article_df['doc_id'] = range(len(article_df))

articles_subset = article_df[['doc_id', 'article_text_raw', 'url', 'titles']]
articles = articles_subset['article_text_raw'].tolist()
metadata = articles_subset[['doc_id', 'url', 'titles']].set_index('doc_id').to_dict(orient='index')


In [37]:
nbits = 2   # encode each dimension with 2 bits
doc_maxlen = 300 # truncate passages at 300 tokens

index_name = f'dsan5400_project_nbits={nbits}'

In [38]:
checkpoint = 'colbert-ir/colbertv2.0'

with Run().context(RunConfig(nranks=1, experiment='DSAN5400')):
    config = ColBERTConfig(doc_maxlen=doc_maxlen, nbits=nbits, kmeans_niters=4)
    indexer = Indexer(checkpoint=checkpoint, config=config)
    logging.info(f"Starting indexing process for {len(articles)} articles...")
    indexer.index(name=index_name, collection=articles, overwrite=True)
    logging.info("Indexing complete.")



[Dec 09, 21:57:55] #> Note: Output directory /content/experiments/DSAN5400/indexes/dsan5400_project_nbits=2 already exists


[Dec 09, 21:57:55] #> Will delete 10 files already at /content/experiments/DSAN5400/indexes/dsan5400_project_nbits=2 in 20 seconds...
#> Starting...
#> Joined...


In [47]:
# To create the searcher using its relative name (i.e., not a full path), set
# experiment=value_used_for_indexing in the RunConfig.
with Run().context(RunConfig(experiment='DSAN5400')):
    searcher = Searcher(index=index_name, collection=articles)


# If you want to customize the search latency--quality tradeoff, you can also supply a
# config=ColBERTConfig(ncells=.., centroid_score_threshold=.., ndocs=..) argument.
# The default settings with k <= 10 (1, 0.5, 256) gives the fastest search,
# but you can gain more extensive search by setting larger values of k or
# manually specifying more conservative ColBERTConfig settings (e.g. (4, 0.4, 4096)).

[Dec 09, 22:04:37] #> Loading codec...
[Dec 09, 22:04:37] #> Loading IVF...
[Dec 09, 22:04:37] #> Loading doclens...


100%|██████████| 1/1 [00:00<00:00, 1780.26it/s]

[Dec 09, 22:04:37] #> Loading codes and residuals...



100%|██████████| 1/1 [00:00<00:00, 16.61it/s]


In [55]:
# Sample query
query = articles[15]
#print(f"#> {query}")

# Find the top-6 passages for this query
results = searcher.search(query, k=6)

# Print out the top-5 retrieved passages excluding the first one (result 1)
for i, (passage_id, passage_rank, passage_score) in enumerate(zip(*results)):
    if i == 0:
        continue

    # Error Handling:
    if passage_id < len(searcher.collection):
        # Get the document ID and corresponding metadata (url, title)
        metadata_info = metadata.get(passage_id, None)

        if metadata_info:
            url = metadata_info['url']
            title = metadata_info['titles']
        else:
            url = "URL not found"
            title = "Title not found"

        # Print the results
        print(f"Rank: {passage_rank - 1}\tScore: {passage_score:.3f}")
        print(f"Title: {title}")
        print(f"URL: {url}")
        print(f"Passage ID: {passage_id}")
        print(f"\n")

    else:
        print(f"Warning: Passage ID {passage_id} is out of range for the collection.")

Rank: 1	Score: 16.484
Title: After Trump’s win, some women are considering the 4B movement
URL: https://www.cnn.com/2024/11/09/us/4b-movement-trump-south-korea-wellness-cec/index.html
Passage ID: 2240


Rank: 2	Score: 15.875
Title: US, South Korea and Japan slam ‘dangerous’ ties between Russia, North Korea
URL: https://www.aljazeera.com/news/2024/11/16/us-south-korea-and-japan-slam-dangerous-ties-between-russia-north-korea
Passage ID: 382


Rank: 3	Score: 15.828
Title: South Korea condemns Russia’s move to seal defence pact with North Korea
URL: https://www.aljazeera.com/news/2024/10/25/south-korea-condemns-russias-move-to-seal-defence-pact-with-north-korea
Passage ID: 554


Rank: 4	Score: 15.711
Title: South Korea says ‘not ruling out’ supplying weapons to Ukraine
URL: https://www.aljazeera.com/news/2024/11/7/south-korea-says-not-ruling-out-supplying-weapons-to-ukraine
Passage ID: 338


Rank: 5	Score: 15.695
Title: RFK Jr., on eve of Election Day, tells voters 'do NOT vote for me'
URL

In [64]:
# Get the index from URL
def get_article_index_by_url(url, metadata):
    for idx, metadata_info in metadata.items():
        if metadata_info['url'] == url:
            return idx
    return None

# Prompt for URL
user_url = input("Please enter the URL of the article: ")

# Find the index from URL
article_index = get_article_index_by_url(user_url, metadata)

# Function with error handling
if article_index is not None:
    query = articles[article_index]
    query_metadata = metadata.get(article_index, None)
    print("Showing top 5 most similar articles to: " + query_metadata['titles'])
    print(f"\n")

    # Perform the search to find the top-k passages
    results = searcher.search(query, k=6)

    # Print out the top-5 retrieved passages excluding the first one (result 1)
    for i, (passage_id, passage_rank, passage_score) in enumerate(zip(*results)):
        if i == 0:
            continue

        if passage_id < len(searcher.collection):
            metadata_info = metadata.get(passage_id, None)

            if metadata_info:
                url = metadata_info['url']
                title = metadata_info['titles']
            else:
                url = "URL not found"
                title = "Title not found"

            print(f"Rank: {passage_rank - 1}")
            print(f"Title: {title}")
            print(f"URL: {url}")
            print(f"Passage ID: {passage_id}")
            print(f"\n")
        else:
            print(f"Warning: Passage ID {passage_id} is out of range for the collection.")
else:
    print("No article found for the given URL.")


Please enter the URL of the article: https://www.aljazeera.com/program/the-stream/2024/11/1/what-issues-are-americans-facing-this-election
Showing top 5 most similar articles to: What issues are Americans facing this election?


Rank: 1	Score: 18.328
Title: Post-election decision: Do you want to live in a red state or a blue state?
URL: https://www.foxnews.com/opinion/post-election-decision-do-you-want-live-red-state-blue-state
Passage ID: 1105


Rank: 2	Score: 18.031
Title: A simple guide to the US election 2024
URL: https://www.aljazeera.com/features/2024/10/30/a-simple-guide-to-the-us-election-2024
Passage ID: 30


Rank: 3	Score: 17.844
Title: Election anxiety leads many Americans to avoid family and friends, survey finds
URL: https://www.foxnews.com/health/election-anxiety-leads-many-americans-avoid-family-friends-survey-finds
Passage ID: 772


Rank: 4	Score: 17.750
Title: The world watches as US election results trickle in
URL: https://apnews.com/article/election-world-watches-pho

/content/ColBERT/colbert/utils/amp.py:15: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  return torch.cuda.amp.autocast() if self.activated else NullContextManager()


In [57]:
searcher.collection[passage_id]

'Former 2024 presidential candidate Robert F. Kennedy, Jr. discusses the close race between former President Trump and Vice President Harris in Michigan, accuses Democrats of trying to \'confuse voters\' on \'The Story.\'\r\nRobert F. Kennedy Jr. is telling Americans "do NOT vote for me" on the eve of Election Day, adding in a video that a vote for Trump is "the only way that we can get me and everything I stand for into Washington, D.C. and fulfill the mission that motivated my campaign."\xa0\r\nThe fresh plea comes after the Supreme Court last week denied separate appeals by RFK Jr. to remove his name from the ballots in Wisconsin and Michigan. \xa0\r\n"I\'ve asked my voters repeatedly to vote for President Trump, but even a few votes for me in [Michigan] could swing the ballot in a way that the public doesn\'t intend and the voters don\'t intend," he told "America\'s Newsroom" on Sunday.\r\nKennedy, who once was running as an independent presidential candidate, has been trying to ge

In [ ]:
results

([414, 29, 60], [1, 2, 3], [23.359375, 22.140625, 21.0625])